In [1]:
import attr
import astropy.units as u
import numpy as np

In [29]:
@attr.s
class galaxy:
    x = attr.ib(validator=attr.validators.instance_of(u.Quantity))
    y = attr.ib(validator=attr.validators.instance_of(u.Quantity))
    z = attr.ib(default = 0.*(u.km/u.s), validator=attr.validators.instance_of(u.Quantity))
    r = attr.ib(default=None)
    
    def __change_units_to_array__(f):
        def new_func(self,*args,**kwargs):    
            self.x = self.x.to_value(u.kpc)
            self.y = self.y.to_value(u.M_sun)
            self.z = self.z.to_value(u.km/u.s)
            return f(self,*args,**kwargs)
        return new_func
    
    @__change_units_to_array__
    def func(self, s=2):
        w = 2* self.x * self.y 
        return w
    
#     @__change_units_to_array__
#     def func2(self):
#         q = 2*self.x
#         return q

In [30]:
yy = galaxy(x=5*u.kpc, y=3*u.M_sun)
yy

galaxy(x=<Quantity 5. kpc>, y=<Quantity 3. solMass>, z=<Quantity 0. km / s>, r=None)

In [31]:
yy.x

<Quantity 5. kpc>

In [32]:
yy.func()

<Quantity 30. kpc solMass>

In [27]:
yy.func2()

<Quantity 10. kpc>

In [28]:
type(yy.x)

astropy.units.quantity.Quantity

In [15]:
5*u.Mpc + 300*u.kpc

<Quantity 5.3 Mpc>

In [17]:
type(np.asarray(y.func()))

numpy.ndarray

In [44]:
type(y.x)

astropy.units.quantity.Quantity

In [43]:
y.x

<Quantity 5000. kpc>

In [25]:
u.ma

AttributeError: module 'astropy.units' has no attribute 'mass'

In [40]:
@u.quantity_input(a=['length', 'speed'])
def myfunction(a):
    return a.to_value(u.kpc)

In [41]:
myfunction(5*u.Mpc)

5000.0

In [85]:
@attr.s
class galaxy:
    x = attr.ib(validator=attr.validators.instance_of(u.Quantity))
    y = attr.ib(validator=attr.validators.instance_of(u.Quantity))
    
    @u.quantity_input(a=['length'])
    def myfunction(self):
        return self.a.to_value(u.kpc)
    
    @u.quantity_input(a=['mass'])
    def myfunction2(self):
        return self.y.to_value(u.Msun)



In [86]:
y=galaxy(y=3e20*u.m,x=5*u.Mpc)

In [100]:
y.x.unit.physical_type


'length'

In [ ]:
y.myfunction()

In [78]:
y.y

<Quantity 3.e+20 m>

In [103]:
y = 3*u.M_sun

In [104]:
y.unit.physical_type

'mass'

In [50]:
G = 4.299e-6 * u.kpc * (u.km / u.s) ** 2 / u.M_sun

In [55]:
G.to_value()

4.299e-06